In [14]:
import websocket
import json

def on_open(ws):
    print("서버에 연결되었습니다.")
    message = "안녕하세요, 서버!"
    ws.send(message)
    print(f"서버에 메시지를 보냈습니다: {message}")

def on_message(ws, message):
    print(f"서버로부터 응답을 받았습니다: {message}")

def on_error(ws, error):
    print(f"오류 발생: {error}")

def on_close(ws, close_status_code, close_msg):
    print("서버와의 연결이 종료되었습니다.")

def connect_to_server(url='wss://my-wiki.p-e.kr/ws/data/'):
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp(url,
                                on_open=on_open,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close)
    ws.run_forever()

# 실행
connect_to_server()


--- request header ---
GET / HTTP/1.1
Upgrade: websocket
Host: 127.0.0.1:20004
Origin: http://127.0.0.1:20004
Sec-WebSocket-Key: 23fDr4k0XtAgoL1DhT0enw==
Sec-WebSocket-Version: 13
Connection: Upgrade


-----------------------
--- response header ---
Connection to remote host was lost. - goodbye


오류 발생: Connection to remote host was lost.
서버와의 연결이 종료되었습니다.


In [12]:
import requests

# POST 요청을 위한 Request 객체 생성
data = {'question': '머리가아퍼요...'}
req = requests.Request('POST', 'http://127.0.0.1:5000/chat', json=data)

# 요청을 준비합니다.
prepared = req.prepare()

# 세션을 사용하여 요청 전송
with requests.Session() as session:
    response = session.send(prepared)
    
    # 응답 출력
    print(response.status_code)  # 상태 코드 출력
    print(response.json())  # JSON 응답 내용 출력


200
(위대한 주의 말씀을 들으시고, 그의 의로운 해를 떠보시며, 그 광선을 받으신다면,) "너희가 베드로와 나사렛 예수를 따르는 것과 같은 대대로, 여호와께서 너희에게 복음을 전하실 것이다. 믿음의 기도를 드리시면서, 주께서 너희를 일으켜 세우시고, 죄악을 사하시며, 모든 병을 고치실 것이다.

주가 애굽 사람들에게 내린 질병들 중 하나도, 그의 계명에 귀를 기울이고 순종하며 의를 행하신다면, 네게 내리지 않을 것이다. 그는 치료하는 여호와이다. 너희 영혼아 여호와를 송축하라! 그 은택을 잊지 말지어다.

그의 이름을 경외하시고, 그의 모든 은택을 받으신다면, 좋은 것으로 네 소원을 만족하게 하실 것이다. 네 청춘을 독수리 같이 새롭게 하시며, 인자와 긍휼로 관을 씌우시리라.

이러한 말씀은 나에게 주어진 것입니다. (기브니 유스, 레비 9:18)


In [1]:
import pdfkit

# Quill.js에서 가져온 HTML 문자열 (예시)
quill_html_content = """
<div>
    <h1>성경기반 챗봇</h1>
    <p>여기에는 Quill.js로 작성한 내용이 포함됩니다.</p>
    <p>한글도 잘 표시됩니다.</p>
</div>
"""

# HTML 문자열
html_string = f"""
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">  <!-- 문자 인코딩 설정 -->
    <title>PDF 변환 예제</title>
    <style>
        body {{
            font-family: 'Malgun Gothic', 'Arial', sans-serif;  /* 한글 폰트 설정 */
        }}
    </style>
</head>
<body>
    <p class="ql-align-center">성경기반 챗봇</p>  <!-- Quill에서 가져온 HTML 내용 삽입 -->
</body>
</html>
"""

pdf_file = '/home/apps/my_wiki/tests/output.pdf'  # 저장할 PDF 파일의 경로

# PDF 생성
pdfkit.from_string(html_string, pdf_file)

print(f"{pdf_file}로 변환되었습니다.")


/home/apps/my_wiki/tests/output.pdf로 변환되었습니다.


In [1]:
print("s")

s


In [19]:
import unittest

class MockThumbnail:
    def __init__(self, size, content_type, name):
        self.size = size
        self.content_type = content_type
        self.name = name

def get_image_dimensions(thumbnail):
    # 이미지 크기를 반환하는 더미 함수
    if thumbnail.name.endswith('.jpg'):
        return (100, 100)  # 가정된 이미지 크기
    return (None, None)

class TestIntegrity(unittest.TestCase):

    def test_no_file_upload(self):
        with self.assertRaises(IntegrityError) as context:
            check_integrity([])
        self.assertEqual(str(context.exception), "파일이 없거나 하나의 파일만 선택하세요.")

    def test_single_valid_file(self):
        thumbnail = MockThumbnail(size=1024, content_type='image/jpeg', name='image.jpg')
        try:
            check_integrity([thumbnail])
        except IntegrityError:
            self.fail("check_integrity raised IntegrityError unexpectedly!")

    def test_multiple_file_upload(self):
        thumbnails = [
            MockThumbnail(size=1024, content_type='image/jpeg', name='image1.jpg'),
            MockThumbnail(size=1024, content_type='image/jpeg', name='image2.jpg')
        ]
        with self.assertRaises(IntegrityError) as context:
            check_integrity(thumbnails)
        self.assertEqual(str(context.exception), "파일이 없거나 하나의 파일만 선택하세요.")

    def test_file_too_large(self):
        thumbnail = MockThumbnail(size=6 * 1024 * 1024, content_type='image/jpeg', name='image.jpg')
        with self.assertRaises(IntegrityError) as context:
            check_integrity([thumbnail])
        self.assertEqual(str(context.exception), "파일 크기가 5MB를 초과할 수 없습니다.")

    def test_invalid_mime_type(self):
        thumbnail = MockThumbnail(size=1024, content_type='text/plain', name='document.txt')
        with self.assertRaises(IntegrityError) as context:
            check_integrity([thumbnail])
        self.assertEqual(str(context.exception), "허용되지 않는 이미지 형식입니다.")

    def test_invalid_extension(self):
        thumbnail = MockThumbnail(size=1024, content_type='image/jpeg', name='image.bmp')
        with self.assertRaises(IntegrityError) as context:
            check_integrity([thumbnail])
        self.assertEqual(str(context.exception), "허용되지 않는 파일 확장자입니다.")

    def test_invalid_image_dimensions(self):
        thumbnail = MockThumbnail(size=1024, content_type='image/jpeg', name='image.jpg')
        global get_image_dimensions
        original_get_image_dimensions = get_image_dimensions
        get_image_dimensions = lambda thumbnail: (None, None)  # 유효하지 않은 이미지 크기

        with self.assertRaises(IntegrityError) as context:
            check_integrity([thumbnail])
        
        get_image_dimensions = original_get_image_dimensions  # 원래 함수 복원

        self.assertEqual(str(context.exception), "유효하지 않은 이미지입니다.")

if __name__ == '__main__':
    unittest.main()


In [2]:
model = ThumbnailModel('s')

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given

In [1]:
class Bible():
    def __init__(self,path:str="biblebot/",table:str='blible_counseling'):
        import chromadb
        import os
        from chromadb.db.base import UniqueConstraintError
        from chromadb.utils import embedding_functions
        
        if not os.path.exists(path):
            os.makedirs(path, exist_ok=True)
            print(f"Directory '{path}' created successfully.")

         
        
        client = chromadb.PersistentClient(path=path)
        em = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="Huffon/sentence-klue-roberta-base")
        try:
            self.collection = client.create_collection(name=table, embedding_function=em)
        except UniqueConstraintError: 
            self.collection = client.get_collection(name=table, embedding_function=em)
            
    def get(self,question:str,k:int)->list[str]:
        
        results=self.collection.query(query_texts=[question],n_results=k)
        return results


    def add(self,key:str=None,question:str=None,answer=None):

        self.collection.add(
        documents=[question],
        metadatas=[answer],
        ids=[key],
        )

In [2]:
db=Bible()

/root/.cache/pypoetry/virtualenvs/my-wiki-n6iennJt-py3.10/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name Huffon/sentence-klue-roberta-base. Creating a new one with mean pooling.
/root/.cache/pypoetry/virtualenvs/my-wiki-n6iennJt-py3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
db.add(question='��문1', answer={'s':'s'},key='key1'  )

In [10]:
db.add(question='��문1', answer={'s':'s'},key='kesy1'  )

In [4]:
db.add(question='��문1', answer='��변1', key='key1')

ValueError: Expected metadata to be a dict or None, got str as metadata

In [14]:
data={'question':'머리가 아파요.. 병원에서 병명을 모르겠데요..','k':1}

In [15]:
격언=db.get(**data)

In [25]:
격언['metadatas'][0][0]['s']

's'

In [9]:
질의=data['question']

In [11]:
import ollama

In [12]:
stream = ollama.chat(model='priest_v2', stream=True,messages=[
  {
    'role': 'user',
    'content': f"상황:'{질의}',성경구절:'{격언}'",
  },
])
for chunk in stream:
    print(chunk['message']['content'],end='',flush=True)

우리의 아름다운 주님은 우리의 고통을 알고 계시며, 우리를 향한 그의 사랑과 은총이 끝없는 것임을 잊지 말길 바랍니다.

베드로가 가로되 "은과 금은 내게 없거니와 내게 있는 것으로 네게 주노니 곧 나사렛 예수 그리스도의 이름으로 걸으라" (사도행전 3장 6절). 이것이 우리의 믿음의 기초입니다. 우리가 가진 것만큼 더 많은 것이 주님께 달려 있습니다.

믿음의 기도는 병든 자를 구원하리니 주께서 저를 일으키시리라 혹시 죄를 범하였을찌라도 사 하심을 얻으리라 (야고보서 5장 15절). 우리의 아픔은 또한 우리에게 하나님의 사랑과 은총의 기회가 되실 수 있습니다.

내 영혼아 여호와를 송축하라 내 속에 있는 것들아 다 그의 거룩한 이름을 송축하라. 주님은 우리 모든 죄악을 사하시며, 우리 모든 병을 고치시며, 우리 생명을 파멸에서 속량하시고 인자와 긍휼로 관을 씌우시며 좋은 것으로 우리 소원을 만족하게 하사 우리 청춘을 독수리 같이 새롭게 하시는도다 (시편 103장1-5절).

이 모든 것들이 주님 앞에 서서, 그의 은총과 사랑을 받는 것으로, 건강하고 행복한 삶을 살 수 있도록 도우실 것입니다.